<h1> Visual Recognition Mini Project
<h4> Varsha Yamsani(IMT2022506)<br>
<h4> Keshav Goyal(IMT2022560)<br>
<h4> R Harshavardhan(IMT2022515)

<h3>Task A and B</h3>
<h5>In this section, we explore two approaches for classifying faces as "with mask" or "without mask." The first method involves extracting handcrafted features and using traditional machine learning classifiers like SVM and Neural Networks. The second approach leverages the power of Convolutional Neural Networks (CNNs) to automatically learn features from images. We will train and evaluate both methods, experiment with hyperparameter tuning, and compare their performance to determine which approach is more effective for face mask classification.</h5>

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


<h3> Loading the dataset

In [ ]:
datasetPath = "dataset"  # Define the main dataset directory
images, labels = [], []  # Initialize lists to store images and corresponding labels

# Loop through both categories: "with_mask" and "without_mask"
for category in ["with_mask", "without_mask"]:  
    path = os.path.join(datasetPath, category)  # Construct the full path to category folder
    label = 1 if category == "with_mask" else 0  # Assign labels: 1 for "with_mask", 0 for "without_mask"

    # Iterate through all image files in the category folder
    for file in os.listdir(path):  
        img_path = os.path.join(path, file)  # Construct full path to the image
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read the image in grayscale
        
        if img is None:  # Handle case where image file could not be read
            print(f"Warning: Could not read {img_path}")
            continue  # Skip to the next image
        
        img = cv2.resize(img, (64, 64))  # Resize image to 64x64 pixels
        images.append(img)  # Append the processed image to the list
        labels.append(label)  # Append the corresponding label  

# Convert lists to NumPy arrays for further processing
X, y = np.array(images), np.array(labels)

# Prepare input data for CNN by normalizing pixel values (0-1) and reshaping for CNN input
xCNN = X.reshape(-1, 64, 64, 1) / 255.0  


In [4]:
import os
import cv2
import numpy as np

# Define dataset path
datasetPath = "dataset"
images, labels = [], []
dataset_structure = {}

# Loop through categories: "with_mask" and "without_mask"
for category in ["with_mask", "without_mask"]:  
    path = os.path.join(datasetPath, category)  # Construct category path
    label = 1 if category == "with_mask" else 0  # Assign labels
    
    if not os.path.exists(path):
        print(f"Warning: Directory {path} does not exist!")
        continue
    
    file_list = os.listdir(path)  # List files in the category folder
    dataset_structure[category] = len(file_list)  # Store category count
    
    for file in file_list:  
        img_path = os.path.join(path, file)  # Full image path
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read in grayscale
        
        if img is None:  # Handle unreadable files
            print(f"Warning: Could not read {img_path}")
            continue  
        
        img = cv2.resize(img, (64, 64))  # Resize image
        images.append(img)  # Store image
        labels.append(label)  # Store label

# Convert lists to NumPy arrays
X, y = np.array(images), np.array(labels)

# Normalize and reshape input for CNN
xCNN = X.reshape(-1, 64, 64, 1) / 255.0

# Print dataset structure
print("Dataset Structure:")
for category, count in dataset_structure.items():
    print(f"{category}: {count} images")
print(f"Total images: {len(X)}")


Dataset Structure:
with_mask: 2165 images
without_mask: 1930 images
Total images: 4095


<h2>Task A: Binary Classification using Handcrafted features and ML Classifiers

<h3>Pre-processing the data for task A, which means extracting handcrafted features and concatenating thmem into a single feature vector to store them in a list for processing. This includes scaling the data and splitting the data into a train and test frames.

In [4]:
features = []  # List to store feature vectors
for img in images:
    # HOG (Histogram of Oriented Gradients) Feature Extraction
    hogFeatures = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)

    # Sobel Edge Detection (Gradient in x and y directions)
    sobelX = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)  # Sobel filter in x-direction
    sobelY = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)  # Sobel filter in y-direction
    sobelCombined = cv2.magnitude(sobelX, sobelY).flatten()  # Compute gradient magnitude and flatten

    # Canny Edge Detection
    canny_edges = cv2.Canny(img, 100, 200).flatten()  # Apply Canny edge detection and flatten

    # Concatenate all features into a single vector
    feature_vector = np.hstack([hogFeatures, sobelCombined, canny_edges])
    features.append(feature_vector)  # Append feature vector to list


In [5]:
xFeatures = np.array(features)  

In [6]:
xTrain, xTest, yTrain, yTest = train_test_split(xFeatures, y, test_size=0.2, random_state=42) #Split the data into training and testing sets

In [7]:
# Standardize features to have zero mean and unit variance
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)  # Fit and transform training data
xTest = scaler.transform(xTest)  # Transform test data using the same scaler

<h3>SVM Model

In [8]:
svm = SVC(kernel="linear")  # Use a linear kernel for SVM
svm.fit(xTrain, yTrain)  # Train the SVM model
yPredSVM = svm.predict(xTest)  # Make predictions on the test set

<h3>MLP Model(Neural Network)

In [9]:
# Train Neural Network (MLP - Multi-Layer Perceptron)
mlp = MLPClassifier(hidden_layer_sizes=(64,32), max_iter=300)  # Define an MLP with 1 hidden layer of 100 neurons
mlp.fit(xTrain, yTrain)  # Train the MLP model
yPredMLP = mlp.predict(xTest)  # Make predictions on the test set

In [10]:
print("SVM Accuracy:", accuracy_score(yTest, yPredSVM))  # Print accuracy of SVM
print("MLP Accuracy:", accuracy_score(yTest, yPredMLP))  # Print accuracy of MLP

print("\nSVM Classification Report:\n", classification_report(yTest, yPredSVM))
print("\nMLP Classification Report:\n", classification_report(yTest, yPredMLP))

SVM Accuracy: 0.884004884004884
MLP Accuracy: 0.9194139194139194

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.87      0.87       366
           1       0.89      0.90      0.90       453

    accuracy                           0.88       819
   macro avg       0.88      0.88      0.88       819
weighted avg       0.88      0.88      0.88       819


MLP Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.88      0.91       366
           1       0.91      0.95      0.93       453

    accuracy                           0.92       819
   macro avg       0.92      0.92      0.92       819
weighted avg       0.92      0.92      0.92       819



<h2>Task B: Using CNN for Binary Classification

<h5>Splitting into train test for CNN and building a CNN model with different hyperparameters, like checking various optimizers, dropout values and learning rates. We also try two activation functions: Sigmoid and tanh to get different results. Various depths of layers and number of epochs were also tried. Here are few of the cases included:

In [12]:
xTrainCNN, xTestCNN, yTrainCNN, yTestCNN = train_test_split(xCNN, y, test_size=0.2, random_state=42)


<h3> Using SIGMOID activation in last layer

In [13]:
def build_cnn(optimizer='adam', dropout_rate=0.5, learning_rate=1e-3):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),  # First convolutional layer
        MaxPooling2D(2,2),  # Pooling to reduce spatial dimensions
        
        Conv2D(64, (3,3), activation='relu'),  # Second convolutional layer
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),  # Third convolutional layer
        MaxPooling2D(2,2),
        
        Flatten(),  # Flatten feature maps into a single vector
        Dense(128, activation='relu'),  # Fully connected layer
        Dropout(dropout_rate),  # Dropout to reduce overfitting
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Unsupported optimizer! Choose 'adam' or 'sgd'.")
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train CNN with different hyperparameters
optimizers = ['adam', 'sgd']  # Optimizer variations
dropoutValues = [0.3, 0.5]  # Dropout variations
learningRates = [1e-3, 1e-4]  # Learning rate variations

for opt in optimizers:
    for dropout in dropoutValues:
        for learning in learningRates:
            print(f"Training CNN with optimizer={opt}, dropout={dropout}, learning_rate={learning}")
            model = build_cnn(optimizer=opt, dropout_rate=dropout, learning_rate=learning)

            # Callbacks: Stop training early if no improvement & adjust learning rate
            callbacks = [
                EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)
            ]

        # Train the model
            model.fit(xTrainCNN,yTrainCNN,  validation_data=(xTestCNN, yTestCNN), epochs=20, callbacks=callbacks, verbose=1)

        # Save the trained model
            model.save(f'face_mask_classifier_{opt}_{dropout}_{learning}.keras')


Training CNN with optimizer=adam, dropout=0.3, learning_rate=0.001


c:\Users\rog\anaconda3\envs\MLTutorial\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.6327 - loss: 0.6183 - val_accuracy: 0.8596 - val_loss: 0.3607 - learning_rate: 0.0010
Epoch 2/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.8578 - loss: 0.3460 - val_accuracy: 0.9219 - val_loss: 0.2189 - learning_rate: 0.0010
Epoch 3/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9049 - loss: 0.2258 - val_accuracy: 0.9402 - val_loss: 0.1695 - learning_rate: 0.0010
Epoch 4/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9344 - loss: 0.1653 - val_accuracy: 0.9243 - val_loss: 0.1965 - learning_rate: 0.0010
Epoch 5/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9422 - loss: 0.1432 - val_accuracy: 0.9512 - val_loss: 0.1385 - learning_rate: 0.0010
Epoch 6/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9634 - loss: 0.1011 - val_accuracy: 0.9341 - val_loss: 0.1637 - learning_rate: 0.0010
Epoch 7/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9752 - loss: 0.

In [14]:
for opt in optimizers:
    for dropout in dropoutValues:
        for learning in learningRates:
            model = keras.models.load_model(f'face_mask_classifier_{opt}_{dropout}_{learning}.keras')
            testLoss, testAccuracy = model.evaluate(xTestCNN, yTestCNN)
            print(f"CNN (Optimizer={opt}, Dropout={dropout}, learning = {learning}) Accuracy: {testAccuracy:.4f}")

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9612 - loss: 0.1386
CNN (Optimizer=adam, Dropout=0.3, learning = 0.001) Accuracy: 0.9609
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9596 - loss: 0.1285
CNN (Optimizer=adam, Dropout=0.3, learning = 0.0001) Accuracy: 0.9597
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9594 - loss: 0.1118
CNN (Optimizer=adam, Dropout=0.5, learning = 0.001) Accuracy: 0.9609
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9691 - loss: 0.1318
CNN (Optimizer=adam, Dropout=0.5, learning = 0.0001) Accuracy: 0.9683
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8055 - loss: 0.4647
CNN (Optimizer=sgd, Dropout=0.3, learning = 0.001) Accuracy: 0.8144
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8412 - loss: 0.3859
CNN (Optimizer=sgd, Dropout=0.3, learning = 0.0001) Accuracy: 0.8608
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8100 - loss: 0.4253
CNN (Optimizer=sgd, Dropout=0.5, learning = 0.001) Accuracy: 0.8303
26/26

<h3>Using tanh in Output layer 

In [15]:
def build_cnn(optimizer='adam', dropout_rate=0.5, learning_rate=1e-3):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),  # First convolutional layer
        MaxPooling2D(2,2),  # Pooling to reduce spatial dimensions
        
        Conv2D(64, (3,3), activation='relu'),  # Second convolutional layer
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),  # Third convolutional layer
        MaxPooling2D(2,2),
        
        Flatten(),  # Flatten feature maps into a single vector
        Dense(128, activation='relu'),  # Fully connected layer
        Dropout(dropout_rate),  # Dropout to reduce overfitting
        Dense(1, activation='tanh')  # Output layer for binary classification
    ])
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Unsupported optimizer! Choose 'adam' or 'sgd'.")
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train CNN with different hyperparameters
optimizers = ['adam', 'sgd']  # Optimizer variations
dropoutValues = [0.3, 0.5]  # Dropout variations
learningRates = [1e-3, 1e-4]  # Learning rate variations

for opt in optimizers:
    for dropout in dropoutValues:
        for learning in learningRates:
            print(f"Training CNN with optimizer={opt}, dropout={dropout}, learning_rate={learning}")
            model = build_cnn(optimizer=opt, dropout_rate=dropout, learning_rate=learning)

            # Callbacks: Stop training early if no improvement & adjust learning rate
            callbacks = [
                EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)
            ]

        # Train the model
            model.fit(xTrainCNN,yTrainCNN,  validation_data=(xTestCNN, yTestCNN), epochs=20, callbacks=callbacks, verbose=1)

        # Save the trained model
            model.save(f'tanh_face_mask_classifier_{opt}_{dropout}_{learning}.keras')


Training CNN with optimizer=adam, dropout=0.3, learning_rate=0.001
Epoch 1/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.5566 - loss: 1.1589 - val_accuracy: 0.7070 - val_loss: 0.5370 - learning_rate: 0.0010
Epoch 2/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.7755 - loss: 0.4627 - val_accuracy: 0.8791 - val_loss: 0.3140 - learning_rate: 0.0010
Epoch 3/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.6865 - loss: 0.9311 - val_accuracy: 0.8266 - val_loss: 0.4184 - learning_rate: 0.0010
Epoch 4/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8438 - loss: 0.4044 - val_accuracy: 0.8840 - val_loss: 0.3019 - learning_rate: 0.0010
Epoch 5/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8716 - loss: 0.4328 - val_accuracy: 0.5678 - val_loss: 0.9485 - learning_rate: 0.0010
Epoch 6/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6778 - loss: 0.6999 - val_accuracy: 0.7668 - val_loss: 0.4630 - learning_rate: 0.0010
Epoch 7/20
103/

In [16]:
for opt in optimizers:
    for dropout in dropoutValues:
        for learning in learningRates:
            model = keras.models.load_model(f'tanh_face_mask_classifier_{opt}_{dropout}_{learning}.keras')
            testLoss, testAccuracy = model.evaluate(xTestCNN, yTestCNN)
            print(f"CNN (Optimizer={opt}, Dropout={dropout}, learning = {learning}) Accuracy: {testAccuracy:.4f}")

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8835 - loss: 0.2973
CNN (Optimizer=adam, Dropout=0.3, learning = 0.001) Accuracy: 0.8840
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9474 - loss: 0.1358
CNN (Optimizer=adam, Dropout=0.3, learning = 0.0001) Accuracy: 0.9548
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9526 - loss: 0.1593 
CNN (Optimizer=adam, Dropout=0.5, learning = 0.001) Accuracy: 0.9487
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9263 - loss: 0.1944
CNN (Optimizer=adam, Dropout=0.5, learning = 0.0001) Accuracy: 0.9341
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8351 - loss: 0.4237
CNN (Optimizer=sgd, Dropout=0.3, learning = 0.001) Accuracy: 0.8352
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8014 - loss: 0.4583
CNN (Optimizer=sgd, Dropout=0.3, learning = 0.0001) Accuracy: 0.8144
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8410 - loss: 0.3514
CNN (Optimizer=sgd, Dropout=0.5, learning = 0.001) Accuracy: 0.8559
26/

<h3>Comparing the three results
<h5> Only the best CNN Model was taken into consideration for comparision with traditional approach

In [17]:
print("SVM Accuracy:", accuracy_score(yTest, yPredSVM))  # Print accuracy of SVM
print("MLP Accuracy:", accuracy_score(yTest, yPredMLP))  # Print accuracy of MLP
model = keras.models.load_model(f'face_mask_classifier_{'adam'}_{0.5}_{0.0001}.keras')
testLoss, testAccuracy = model.evaluate(xTestCNN, yTestCNN)
print(f"CNN (optimizer={'adam'}, Dropout={0.5}, learning = {0.0001}) Accuracy: {testAccuracy:.4f}") #Print accuracy of CNN

SVM Accuracy: 0.884004884004884
MLP Accuracy: 0.9194139194139194
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9691 - loss: 0.1318
CNN (optimizer=adam, Dropout=0.5, learning = 0.0001) Accuracy: 0.9683
